In [2]:
'''
import numpy as np
import cv2 as cv
import argparse
import os.path
import time

#Video File

video = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/mr_cutted.mp4'

cap = cv.VideoCapture(video)
hasFrame, frame = cap.read()

outputFile_2 = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/output_3.mp4'

W_in = 1920
H_in = 1080

'''
'''
vid_writer = cv.VideoWriter(outputFile,
                            cv.VideoWriter_fourcc('M','J','P','G'),
                            60,
                            (frame.shape[1], frame.shape[0])
                            )
'''
'''

output = cv.VideoWriter(outputFile_2, cv.VideoWriter_fourcc(*'MP4V'),
                         25, (W_in, H_in))


protoFile = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/colorization_deploy_v2.prototxt'
weightsFile = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/colorization_release_v2.caffemodel'

pts_in_hull = np.load('/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/pts_in_hull.npy')

net = cv.dnn.readNetFromCaffe(protoFile, weightsFile)

pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

timer = []

while cv.waitKey(1):

  hasFrame, frame = cap.read()
  frameCopy = np.copy(frame)

  if not hasFrame:
    break

  start = time.time()

  img_rgb = (frame[:, :, [2, 1, 0]] * 1.0 / 255).astype(np.float32)
  img_lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2Lab)
  #cv2_imshow(img_lab)
  img_l = img_lab[:, :, 0]  # pull out L channel

  # resize lightness channel to network input size
  img_l_rs = cv.resize(img_l, (W_in, H_in))
  img_l_rs -= 50  # subtract 50 for mean-centering

  net.setInput(cv.dnn.blobFromImage(img_l_rs))
  ab_dec = net.forward()[0, :, :, :].transpose((1, 2, 0))  # this is our result

  (H_orig,W_orig) = img_rgb.shape[:2]  # original image size
  ab_dec_us = cv.resize(ab_dec, (W_orig, H_orig))
  img_lab_out = np.concatenate((img_l[:, :, np.newaxis], ab_dec_us), axis=2)  # concatenate with original L channel
  img_bgr_out = np.clip(cv.cvtColor(img_lab_out, cv.COLOR_Lab2BGR), 0, 1)

  end = time.time()
  timer.append(end - start)

  #cv2_imshow((img_bgr_out * 255).astype(np.uint8))

  #vid_writer.write((img_bgr_out * 255).astype(np.uint8))
  output.write((img_bgr_out * 255).astype(np.uint8))

cap.release()
output.release()
#vid_writer.release()

print("Time taken : {:0.5f} secs".format(sum(timer)))
print('Colorized video saved as ' + outputFile_2)
print('Done !!!')
'''


from google.colab.patches import cv2_imshow
import numpy as np
import cv2 as cv
import argparse
import os.path
import time

# Read the input video

video = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/dream_cut_2.mp4'
cap = cv.VideoCapture(video)
hasFrame, frame = cap.read()

W_in = int(1920)
H_in = int(1080)

outputFile = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/output_3.mp4'
output = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc(*'MP4V'),
                         24, (W_in, H_in))

# Specify the paths for the 2 model files
protoFile = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/colorization_deploy_v2.prototxt'
weightsFile = '/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/colorization_release_v2.caffemodel'

pts_in_hull = np.load('/content/drive/MyDrive/ColabNotebooks/computerVision/colorization/pts_in_hull.npy')

# Read the network into Memory
net = cv.dnn.readNetFromCaffe(protoFile, weightsFile)

# populate cluster centers as 1x1 convolution kernel
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

# from opencv sample
#W_in = 224
#H_in = 224

timer = []

while cv.waitKey(1):

    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)
    if not hasFrame:
        break

    start = time.time()

    img_rgb = (frame[:, :, [2, 1, 0]] * 1.0 / 255).astype(np.float32)
    img_lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2Lab)
    img_l = img_lab[:, :, 0]  # pull out L channel

    # resize lightness channel to network input size
    img_l_rs = cv.resize(img_l, (W_in, H_in))
    img_l_rs -= 50  # subtract 50 for mean-centering

    net.setInput(cv.dnn.blobFromImage(img_l_rs))
    ab_dec = net.forward()[0, :, :, :].transpose((1, 2, 0))  # this is our result

    (H_orig,W_orig) = img_rgb.shape[:2]  # original image size
    ab_dec_us = cv.resize(ab_dec, (W_orig, H_orig))
    img_lab_out = np.concatenate((img_l[:, :, np.newaxis], ab_dec_us), axis=2)  # concatenate with original L channel
    img_bgr_out = np.clip(cv.cvtColor(img_lab_out, cv.COLOR_Lab2BGR), 0, 1)

    end = time.time()
    timer.append(end - start)
    #cv2_imshow((img_bgr_out * 255).astype(np.uint8))
    output.write((img_bgr_out * 255).astype(np.uint8))

output.release()
cv.destroyAllWindows()